In [25]:
import cv2
import numpy as np
import os

# Open all the video folders to find the paths
video_folder = "/home/persie/Videos/mokap/"
session_id = os.listdir(video_folder)
video_file = []
for folder in session_id:
    files = os.listdir(os.path.join(video_folder, folder))
    for file in files:
        video_file.append(os.path.join(video_folder, folder, file))

def get_frame(video_path: int, frame_index: int) -> None:
    vid = cv2.VideoCapture(video_path)
    vid.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    ret, frame = vid.read()
    vid.release()
    if not ret:
        return None
    else:
        return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
def draw_circle(event, x, y, flags, param):
    #Because of the use of global variables, they must be defined before using them, outside of any functions
    global radius, text_image, mask
    if flags == cv2.EVENT_FLAG_SHIFTKEY:
        cv2.circle(mask, (x, y), radius, (0, 0, 0), -1)
    if flags == cv2.EVENT_FLAG_LBUTTON:
        cv2.circle(mask, (x, y), radius, (255, 255, 255), -1)

    text_image = cv2.rectangle(text_image, (0, 70), (35, 40), (255, 255, 255), -1)
    text_image = cv2.putText(text_image, str(radius), (0, 65), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8,
                             color=(0, 0, 255), thickness=2)

def create_bgs(video_path, clearest_frame):

    cap = cv2.VideoCapture(video_path)    
    # Read the first frame, but discard as first frame is blank
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read video")
        return
    
    back_sub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=30, detectShadows=True)
    
    back_sub.apply(clearest_frame, None, 1)
  
    return back_sub


    




In [17]:
# For video 33 (for example) select a clear frame to use as the background
video_id = 33
clear_frame_id = 134
object_frame_id = 1507

In [18]:
clear_frame = get_frame(video_file[video_id], clear_frame_id)
object_frame = get_frame(video_file[video_id], object_frame_id)

In [23]:
# Create a mask of the object being tracked, and scan it for good features to track
text_image = object_frame
mask = np.zeros_like(text_image, np.uint8)
# Set the initial circle radius
radius = 50
line_1 = "Click with the left mouse button to add, hold SHIFT to remove"
line_2 = "Up = radius + 5,"
line_3 = "Down = radius - 5"
text_image = cv2.putText(text_image, line_1, (10, 10), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 255, 0),
                         thickness=2)
text_image = cv2.putText(text_image, line_2, (10, 22), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 255, 0),
                         thickness=2)
text_image = cv2.putText(text_image, line_3, (10, 34), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 255, 0),
                         thickness=2)
combined_image = cv2.bitwise_or(text_image, mask)
cv2.namedWindow('Mask On Frame')
cv2.imshow('Mask On Frame', combined_image)
cv2.setMouseCallback('Mask On Frame', draw_circle)

while (1):
    combined_image = cv2.bitwise_or(text_image, mask)
    cv2.imshow('Mask On Frame', combined_image)
    cv2.imshow('Mask', mask)
    k = cv2.waitKey(20) & 0xFF
    if k == ord("q"):
        break
    if k == 82:
        #upkey
        radius = radius + 5
    if k == 84:
        #downkey
        radius = radius - 5

cv2.destroyAllWindows()
    

In [24]:
#Save the mask if you like it
object_mask = mask

In [ ]:
#Find the useful features in the masked image

In [26]:
# Create a background subtractor with the clear frame
bgs = create_bgs(video_file[video_id], clear_frame)

In [15]:

    
    
    #Search the foreground for the features
    
    #Use these matched features as markers for the edge detection method

In [ ]:
#Background Update
def segment_seeds(video_path, bgs):

    cap = cv2.VideoCapture(video_path)
    # Read the first frame
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read video")
        return
    kernel = np.ones((3,3),np.uint8)
    
    
    while True:
        #For each frame in the video
        ret, frame = cap.read()
        if not ret:
            break

        # Convert current frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        #Use the background subtractor to estimate what is in the foreground
        fg_mask = bgs.apply(gray, None, 0.0)
        # noise removal
        # This is the sure fg as it is 0 where the scene is the background
        sure_fg = cv2.threshold(fg_mask, 0, 255, cv2.THRESH_BINARY)[1]
        sure_fg = cv2.erode(sure_fg, kernel, iterations=1)
        sure_fg = cv2.dilate(sure_fg, kernel, iterations=1)
        
        #Extract the pixels that are covered by the sure foreground mask
        
        #Search the masked image for features
        
        #Show the matched features



        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
